# Use bulkInsert to test GPU index

These instructions are adpated from [Zilliz](https://zilliverse.feishu.cn/wiki/ZWtMw4Lo3iDIvqkXjB0cM4ISnUW) wiki instructions.

## Env preparation

1. Docker installation: https://docs.docker.com/engine/install/ubuntu/
2. Install nvidia-docker2:

In [ ]:
curl -s -L https://nvidia.github.io/nvidia-docker/gpgkey | sudo apt-key add -
distribution=$(. /etc/os-release;echo $ID$VERSION_ID)
curl -s -L https://nvidia.github.io/nvidia-docker/$distribution/nvidia-docker.list | sudo tee /etc/apt/sources.list.d/nvidia-docker.list
sudo apt-get update
sudo apt-get install nvidia-docker2
sudo systemctl restart docker.service

3. Install NVIDIA driver

In [ ]:
sudo apt install --no-install-recommends  nvidia-headless-535 nvidia-utils-535

4. (Optional) mount a high performance disk for test. We need to ensure all following operations are in a high performance disk. For an AWS host, we need to manually mount the NVMe SSD (For example, g4dn).

In [ ]:
lsblk # see device path
sudo mkfs -t ext4 /dev/nvme1n1
sudo mkdir /data
sudo mount /dev/nvme1n1 /data
sudo -i blkid # get /dev/nvme1n1 UUID, e.g. dd04113f-deb6-42b0-a021-03110c119295 
sudo vi /etc/fstab # add to the tail: UUID=<UUID get from previous cmd> /data ext4 defaults 1 2
cd /data

## Download milvus image

In [ ]:
sudo docker pull milvusdb/milvus:v2.4.0-rc.1

## Use docker compose to start the milvus service 

1. Save the following file as docker-compose.yml.

2. Start Milvus by running `docker compose`: 

In [ ]:
sudo docker compose up -d